In [1]:
import numpy as np
from scipy.stats            import entropy
from scipy.spatial.distance import squareform
import os

import pyximport
pyximport.install()

(None, <pyximport.pyximport.PyxImporter at 0x7f53b02cb550>)

In [2]:
import jsdiv
dat_dir = "/data/clintko/SMPK"

In [3]:
jsdiv

<module 'jsdiv' (/home/clintko/.pyxbld/lib.linux-x86_64-3.6/jsdiv.cpython-36m-x86_64-linux-gnu.so)>

In [4]:
jsdiv.pairwise_jsdiv_cy

<function jsdiv.pairwise_jsdiv_cy>

In [5]:
p_raw = np.load(os.path.join(
    dat_dir, 
    "ep8cs_tsne_similarity" + "_raw.npy"))

In [6]:
%%time
mat_raw2 = np.zeros((p_raw.shape[1], p_raw.shape[1]))
jsdiv.pairwise_jsdiv_cy(mat_raw2, p_raw, p_raw)

CPU times: user 4min 17s, sys: 1.19 s, total: 4min 18s
Wall time: 4min 17s


In [7]:
mat_raw2[:2, :2]

array([[ 0.        , 11.46403666],
       [11.46403666,  0.        ]])

In [12]:
import glob,re

In [13]:
fnames = glob.glob(os.path.join(dat_dir, "ep8cs_dat_cal_prior*"))
priors = [re.findall("prior(.*).txt", x)[0] for x in fnames]

In [14]:
len(priors)

18

In [68]:
pairwise_jsdiv1(P1, P2)

array([[0.74373885, 0.72014541, 0.63822818, 1.02530635, 0.82888757],
       [0.28597324, 1.03869919, 0.87391215, 1.24839417, 1.51509191],
       [1.5138792 , 0.9909233 , 1.10772133, 1.06616801, 0.73915728],
       [0.65030499, 0.4951706 , 1.13909117, 1.33976234, 0.84937533],
       [0.74071711, 1.08445857, 1.52885035, 1.37445163, 1.32514123]])

In [77]:
vec = pairwise_jsdiv2(P1, P2)
mat = np.zeros((5, 5))
idx = np.triu_indices(5)
mat[idx] = vec
symmetrize(mat)

array([[0.74373885, 0.72014541, 0.63822818, 1.02530635, 0.82888757],
       [0.72014541, 1.03869919, 0.87391215, 1.24839417, 1.51509191],
       [0.63822818, 0.87391215, 1.10772133, 1.06616801, 0.73915728],
       [1.02530635, 1.24839417, 1.06616801, 1.33976234, 0.84937533],
       [0.82888757, 1.51509191, 0.73915728, 0.84937533, 1.32514123]])

In [74]:
def kldiv(P1, P2):
    """calculate KL divergence"""
    P1 = np.array(P1)
    P2 = np.array(P2)
    return entropy(P1.ravel(), P2.ravel())

def jsdiv(P1, P2):
    """calculate Jensen-Shannon Divergence"""
    return 0.5 * (kldiv(P1, P2) + kldiv(P2, P1))

def pairwise_jsdiv1(mat_p1, mat_p2):
    """
    Arg:
        mat_p1, mat_p2: similarity matrix
    """ 
    ### loop over pairwise columns
    n   = mat_p1.shape[1]
    mat_out = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            p1 = mat_p1[:, i]
            p2 = mat_p2[:, j]
            mat_out[i, j] = jsdiv(p1, p2)
    return mat_out

def pairwise_jsdiv2(mat_p1, mat_p2):
    """
    Arg:
        mat_p1, mat_p2: similarity matrix
    """ 
    ### get & check input matrices' dimension
    assert mat_p1.shape[1] == mat_p2.shape[1], "Two input matrices not having same number of columns"
    n = mat_p1.shape[1]
    
    ### init
    out = list()
    ids = np.triu_indices(n)

    ### loop over pairwise columns
    for i, j in zip(*ids):
        p1 = mat_p1[:, i]
        p2 = mat_p2[:, j]
        out.append(jsdiv(p1, p2))
    return out

def symmetrize(mat):
    """make the matrix into symmetric matrix"""
    return mat + mat.T - np.diag(np.diag(mat))